## Dataset for second round of lemur experiments

This notebook takes random image pairs from videos, and saves them as a new dataset. An image pair are two frames which are either directly consecutive or within a few frames from each other.
The number of image pairs taken from a video depends on the estimated number of monkeys in the video.

In [1]:

import os
import sys
if '../../.conda/envs/mktrack/lib/python3.8/site-packages' not in sys.path:
    sys.path.insert(0, '../../.conda/envs/mktrack/lib/python3.8/site-packages')
    sys.path.insert(0, '../src')
    sys.path.insert(0, '../src/lib')
    
import re
import numpy as np
import cv2
import torch
from random import randrange
import matplotlib.pyplot as plt

from models.model import create_model, load_model
from models.decode import mot_decode, map2orig
from utils.post_process import ctdet_post_process
from utils.image import get_affine_transform
from models.utils import _tranpose_and_gather_feat
from datasets.dataset import letterbox


In [2]:
#This function estimates the number of monkeys for a given frame

def estimate_monkeys(frame_num):

    cap.set(cv2.CAP_PROP_POS_FRAMES, frame_num)
    ret, img0 = cap.read()


    # Scale and fill grey borders
    img_box, _, _, _ = letterbox(img0, height=608, width=1088)

    # Normalize RGB
    img = img_box[:, :, ::-1].transpose(2, 0, 1)
    img = np.ascontiguousarray(img, dtype=np.float32)
    img /= 255.0

    im_blob = torch.from_numpy(img).cuda().unsqueeze(0)

    width = img0.shape[1]
    height = img0.shape[0]
    inp_height = im_blob.shape[2]
    inp_width = im_blob.shape[3]
    
    with torch.no_grad():
        output = model(im_blob)[-1]
        hm = output['hm'].sigmoid_()
        wh = output['wh']
        reg = output['reg']

            

    dets, inds, cls_inds_mask = mot_decode(heatmap=hm,
                                                   wh=wh,
                                                   reg=reg,
                                                   num_classes=2,
                                                   cat_spec_wh=False,
                                                   K=50)

    

    dets = map2orig(dets, inp_height, inp_width, height, width, 1)

    lemur_dets = dets[0]

    # remove detections below the threshold

    remain_inds = lemur_dets[:, 4] > 0.4

    total = remain_inds.sum()
    
    return(total)

In [3]:
#!mkdir /usr/users/agecker/datasets/macaque_images

In [4]:
path = "/usr/users/vogg/sfb1528s3/B06/2023april-july/NewBoxesHalfOpen/"

output_path = "/usr/users/vogg/Labelling/Lemurs/LemurBoxApr23/images/"

In [5]:
mp4_list = []
for item in sorted(os.listdir(path)):
  if not item.startswith("."):
    for subitem in os.listdir(path + item):
      if not subitem.startswith("."):
        for subsubitem in os.listdir(path+item+"/"+subitem):
          if subsubitem.endswith(".MP4"):
              mp4_list.append(item+"/"+subitem+"/"+subsubitem)

mp4_list = [elem for elem in mp4_list if ("GX01" in elem) and not ("._" in elem)]
mp4_list[:20]

mp4_list[50:60]
#mp4_list = list(set(mp4_list) - set(rm_list))

['R1/2023-05-08/GX010032_cam3.MP4',
 'R1/2023-05-08/GX010032_cam6.MP4',
 'R1/2023-05-08/GX010032_cam7.MP4',
 'R1/2023-05-08/GX010033_cam1.MP4',
 'R1/2023-05-08/GX010033_cam2.MP4',
 'R1/2023-05-08/GX010034_cam8.MP4',
 'R1/2023-05-17/GX010039_cam9.MP4',
 'R1/2023-05-17/GX010040_cam3.MP4',
 'R1/2023-05-17/GX010040_cam4.MP4',
 'R1/2023-05-17/GX010041_cam1.MP4']

In [6]:
model = create_model('hrnet_32', heads =  {'hm': 2, 'wh': 2, 'id': 128, 'reg': 2}, 
                     head_conv = 256, num_gc_cls=None, clsID4GC=None, gc_with_roi=False)

model = load_model(model, '../models/hrnet32_lemur_sep22.pth')
model = model.to(torch.device('cuda'))
model.eval();

{'hm': 2, 'wh': 2, 'id': 128, 'reg': 2}
loaded ../models/hrnet32_lemur_sep22.pth, epoch 300


In [7]:

for n, video in enumerate(mp4_list[53:]):

    cap = cv2.VideoCapture(path + video)
    print(f"Video {video} loaded")
    #Total number of frames
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    #width  = cap.get(cv2.CAP_PROP_FRAME_WIDTH)   # float `width`
    #height = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)  # float `height`
    

    #we will pull max_monkeys frames out of the video
    samples = 1
    output_name = video.replace('/', '-').split('.')[0]

    while samples < 21:

        try:
            frame = randrange(frame_count)
            n_monkeys = estimate_monkeys(frame)
            if n_monkeys > 0:
                cap.set(cv2.CAP_PROP_POS_FRAMES, frame)
                ret, img0 = cap.read()
                cv2.imwrite(f"{output_path}{output_name}_{str(samples).zfill(5)}.jpg", img0)
                samples = samples + 1
        
        except:
            print("Something went wrong")

    cap.release()

        

Video R1/2023-05-08/GX010033_cam1.MP4 loaded
Video R1/2023-05-08/GX010033_cam2.MP4 loaded
Video R1/2023-05-08/GX010034_cam8.MP4 loaded
Video R1/2023-05-17/GX010039_cam9.MP4 loaded
Video R1/2023-05-17/GX010040_cam3.MP4 loaded
Video R1/2023-05-17/GX010040_cam4.MP4 loaded
Video R1/2023-05-17/GX010041_cam1.MP4 loaded
Video R1/2023-05-17/GX010041_cam6.MP4 loaded
Video R1/2023-05-17/GX010042_cam2.MP4 loaded
Video R1/2023-05-17/GX010042_cam8.MP4 loaded
Video R1/2023-05-17/GX010043_cam7.MP4 loaded
Video R1/2023-05-18/GX010043_cam9.MP4 loaded
Video R1/2023-05-18/GX010045_cam1.MP4 loaded
Video R1/2023-05-18/GX010045_cam3.MP4 loaded
Video R1/2023-05-18/GX010045_cam4.MP4 loaded
Video R1/2023-05-18/GX010045_cam6.MP4 loaded
Video R1/2023-05-18/GX010046_cam2.MP4 loaded
Video R1/2023-05-18/GX010046_cam8.MP4 loaded
Video R1/2023-05-18/GX010047_cam7.MP4 loaded
Video alpha/2023-05-07/GX010028_cam4.MP4 loaded
Video alpha/2023-05-07/GX010028_cam9.MP4 loaded
Video alpha/2023-05-07/GX010029_cam3.MP4 loaded
V